# Music and iteration

Part of the power of computer programming is telling a computer to do the same instructions over and over, but changing something each time. This is the power of iteration, which typically comes in two flavours

1. `for` loop: iterate over a finite collection of things or until a condition is met
2. `while` loop: do something until a condition is met


For our purposes we are going to mainly look at the `for` loop.

We can demonstrate the utility of `for` in two dimensions of music harmony and rhythm.



## Setup

Lets begin by gathering the materials we will need. Some maths functions and a way to quickly listen to audio

In [ ]:
from math import sin, pi
from IPython.display import Audio


## Iteration and time

Let say we wanted to make a sine wav we will need first our frequency $f_0$, our sampling rate, our duration

In [ ]:
f0 = 440.0     # Fundamental frequency
duration = 1.0 # in seconds
fs = 44100.0   # Sampling Rate 



We also then need to figure out how much we need to change angle of the wave for it to be the correct frequency

In [ ]:
delta = 2.0 * pi * f0 / fs # how much does the phase change between samples

A digital sine wav is made up pf many samples, it would be maddening if we had to write

In [ ]:
sine_wave_sample_0  = sin(delta * 0)
sine_wave_sample_1  = sin(delta * 1)
sine_wave_sample_2  = sin(delta * 2)
sine_wave_sample_3  = sin(delta * 3)
sine_wave_sample_4  = sin(delta * 4)
sine_wave_sample_5  = sin(delta * 5)
sine_wave_sample_6  = sin(delta * 6)
sine_wave_sample_7  = sin(delta * 7)
sine_wave_sample_8  = sin(delta * 8)
sine_wave_sample_9  = sin(delta * 9)
sine_wave_sample_10 = sin(delta * 10)

we'd have to do that another few thousand times before we had eough samples to listen to.

Instead we can use a `for` loop make the list of samples for us.

First we figure out how many samples we need. Which just our duration $\times$ our sample rate.

It can only be an integer as we can't have a fraction of a sample

In [ ]:
num_samples = int(duration*fs)

Next we create a list to which we can add our samples

In [ ]:
sine_wave = []

Then our `for` loop, which states thats an index, `i`, is going step through the numbers in a range of numbers between `0` and `num_samples`. That range is provided by the `range` function

In [ ]:
for i in range(num_samples):
    sine_wave.append([sin(delta * i)])


We can actually write this in a slight more nicer way by making the list directly.

If we wanted a list of numbers from `0` to `10` we could write

In [ ]:
[number for number in range(10)]

The same applies to our sine wave as we could jiust as well write

In [ ]:
sine_wave = [sin(delta * i) for i in range(num_samples)]

Audio(data=sine_wave, rate=fs)

## Iteration and harmonics

now maybe we want to play a fifth interval above that, or $\frac{3}{2}f_0$ for our fundamental frequency $f_0$

In [ ]:
f1 = 3/2 * f0
delta = 2.0 * pi * f1 / fs # how much does the phase change between samples
sine_wave_1 = [sin(delta * i) for i in range(int(duration*fs))]
Audio(data=sine_wave_1, rate=fs)

we needed to write slightly less, but we still need to add the two tones together. We can use iteration for that as well. We can zip them together with the `zip` function

In [ ]:
both_sine_waves = [samp1+samp2 for samp1,samp2 in zip(sine_wave,sine_wave_1)]

Audio(data=both_sine_waves, rate=fs)

But what if we want a third sine wave? or a fourth? twenty? All of sudden this approach doesn't scale very well.

That's where we can add another `for` loop.

In [ ]:
harmonics = []

num_harmonics = 10

for i in range(num_samples):
    harmonics.append(0.0)
    for k in range(num_harmonics):
        delta = 2.0 * pi * f0 * k / fs 
        harmonics[i] += sin(delta * i)
    

That is probably pretty loud, but we can check

In [ ]:
max(harmonics)

That is far too loud, so next we normalise the audio so it is in the range `-1.0` > `+1.0`

In [ ]:
maximum = max(harmonics)

for i in range(num_samples):
    harmonics[i] *= 1.0 / maximum
    
Audio(data=harmonics, rate=fs)

Scaling the amplitude of the harmonic inverse to its frequency should result in something a little more pleasant.

Adding in the line

```py
gain = 1.0 / (f0 * k)
```

In [ ]:
harmonics = []

num_harmonics = 10

for i in range(num_samples):
    harmonics.append(0.0)
    for k in range(num_harmonics):
        gain = 1.0 / (f0 * (k+1))
        delta = 2.0 * pi * f0 * k / fs 
        harmonics[i] += gain*sin(delta * i)
        
maximum = max(harmonics)

for i in range(num_samples):
    harmonics[i] *= 1.0 / maximum
    
Audio(data=harmonics, rate=fs)

The `range` also allows for the starting index and step size to be changed, e.g. `range(0,10,2)` instructs to start on `0`, go up to (but not including) `10` and increase in steps of `2`

By just changing the number and step of the harmonics, we can change the timbre of our sound

For a square wave we would only want the odd numbers

In [ ]:
harmonics = []

num_harmonics = 10

for i in range(num_samples):
    harmonics.append(0.0)
    for k in range(1,num_harmonics,2):
        gain = 1.0 / (f0 * (k+1))
        delta = 2.0 * pi * f0 * k / fs 
        harmonics[i] += gain*sin(delta * i)
        
maximum = max(harmonics)

for i in range(num_samples):
    harmonics[i] *= 1.0 / maximum
    
Audio(data=harmonics, rate=fs)